In [57]:
import pandas as pd
from pathlib import Path
import re

st = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AS": "American Samoa",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District Of Columbia",
    "FM": "Federated States Of Micronesia",
    "FL": "Florida",
    "GA": "Georgia",
    "GU": "Guam",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MH": "Marshall Islands",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "MP": "Northern Mariana Islands",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PW": "Palau",
    "PA": "Pennsylvania",
    "PR": "Puerto Rico",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VI": "Virgin Islands",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

def replace_st(x):
    for key, value in st.items():
        x = x.replace(key, value)
    return x

#dfp = None
#pathlist = Path('.').glob('co-est2019-annres-*.xlsx')
#for path in pathlist:
#    df = pd.read_excel(path, header=3)
#    if dfp is None:
#        dfp = df
#    else:
#        dfp = pd.concat([dfp, df])
        
dfp = pd.read_excel('./co-est2019-annres.xlsx', header=1)

df = pd.read_csv('county_adjacency2010.csv')

df['countynameorg'] = df['countyname']
df['countyname'] = df['countyname'].apply(replace_st)
df['neighborname'] = df['neighborname'].apply(replace_st)

dfp['Unnamed: 0'] = dfp['Unnamed: 0'].apply(lambda x : re.sub('^.', '', x))

df = df.merge(dfp, how='left', left_on='countyname', right_on='Unnamed: 0')

df = df[~df['countyname'].str.endswith('Alaska')]
df = df[~df['countyname'].str.endswith('Puerto Rico')]
df = df[~df['countyname'].str.endswith('Virgin Islands')]
df = df[~df['countyname'].str.endswith('American Samoa')]
df = df[~df['countyname'].str.endswith('Guam')]
df = df[~df['countyname'].str.endswith('Northern Mariana Islands')]

df.to_csv('test.csv')

df['same'] = df.apply(lambda x : x['countyname'] == x['neighborname'], axis=1)
df = df[df['same'] == False]

#df.set_index('countyname', inplace=True)

df = df[df[2019].notnull()]

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(df[df['Census'].isnull()])

In [58]:
dfa = df.groupby('fipscounty').agg({
    'countyname': 'first',
    'countynameorg': 'first',
    2019: 'first'
})

sizes = []
n = 0
for index, row in dfa.iterrows():
    sizes.append(str(n))
    n += int(row[2019]/10)
print(', '.join(sizes))

0, 5586, 27909, 30377, 32616, 38398, 39408, 41352, 52712, 56037, 58656, 63098, 64356, 66718, 68041, 69532, 74766, 80290, 81496, 82562, 86266, 87643, 96019, 100936, 104655, 111806, 119926, 123589, 133815, 135445, 138581, 141208, 142019, 143484, 145204, 155792, 160954, 226811, 228191, 237463, 240755, 257209, 267100, 268072, 269878, 307168, 309054, 312024, 321701, 363022, 365095, 387743, 399710, 400602, 402595, 405906, 408178, 413974, 422925, 444695, 445937, 453934, 457970, 478905, 485257, 486889, 487926, 490288, 497476, 510068, 524415, 529816, 533699, 534648, 536758, 985299, 1006517, 1017609, 1122336, 1168614, 1173263, 1196772, 1218150, 1219898, 1221863, 1226056, 1253970, 1257713, 1258789, 1259307, 1262145, 1263156, 1265388, 1266843, 1269334, 1270129, 1272474, 1274558, 1285591, 1291916, 1296711, 1298352, 1299052, 1300188, 1302009, 1314609, 1316380, 1317627, 1327565, 1329391, 1333923, 1336076, 1339453, 1340773, 1344555, 1345917, 1347588, 1354270, 1356927, 1357589, 1359229, 1360114, 136141

In [59]:
n

32624540

In [60]:
neightbors = []
dfa.set_index('countyname', inplace=True)
for index, row in dfa.iterrows():
    ns = []
    for index, row in df[df['countyname'] == index].iterrows():
        try:
            a = dfa.index.get_loc(row['neighborname'])
            if isinstance(a, int):
                ns.append(str(a))
        except:
            pass
    neightbors.append('{ ' + ', '.join(ns) + ' }')
print(', '.join(neightbors))

{ 10, 23, 25, 42, 50 }, { 12, 26, 48, 49, 64, 306 }, { 5, 22, 33, 54, 56, 387, 475, 485 }, { 10, 32, 36, 52, 58, 62 }, { 21, 27, 36, 47, 57, 63 }, { 2, 43, 50, 54, 56 }, { 17, 19, 20, 42, 49, 65 }, { 9, 14, 27, 57, 60 }, { 40, 55, 61, 429, 498 }, { 7, 14, 24, 27, 384, 414, 472 }, { 0, 3, 18, 23, 25, 52, 58 }, { 12, 45, 59, 64, 1380, 1406, 1445 }, { 1, 11, 45, 49, 64, 65 }, { 14, 18, 55, 60, 61 }, { 7, 9, 13, 55, 60, 379, 428, 472 }, { 19, 20, 22, 30, 54 }, { 29, 38, 39, 1439 }, { 6, 19, 26, 49 }, { 10, 13, 25, 58, 60, 61 }, { 6, 15, 17, 20, 26, 30, 336, 356 }, { 6, 15, 19, 42, 50, 54 }, { 4, 39, 47, 51, 63, 66 }, { 2, 15, 30, 33, 34, 54 }, { 0, 10, 42, 45, 52, 65 }, { 9, 27, 35, 47, 384, 398, 503 }, { 0, 10, 18, 43, 50, 61 }, { 1, 17, 19, 49, 306, 336, 347 }, { 4, 7, 9, 24, 47, 57 }, { 37, 46, 53, 62, 63 }, { 16, 39, 46, 66, 1397, 1439 }, { 15, 19, 22, 34, 319, 321, 356 }, { 32, 45, 53, 59, 62 }, { 3, 31, 45, 52, 62 }, { 2, 22, 34, 387, 406 }, { 22, 30, 33, 321, 406, 482 }, { 24, 44, 4

In [61]:
#dfa.set_index('countyname', inplace=True)

from lxml import etree
doc = etree.parse('../src/Usa_counties_large.svg')
root = doc.getroot()

a = {}
for index, row in dfa.iterrows():
    b = row['countynameorg'].replace(' County', '').replace(' Parish', '').replace("'", ' ')
    n = dfa.index.get_loc(index)
    if isinstance(n, int):
        a[b] = {
            'n': n,
            'pop': int(row[2019]/10)
        }
    else:
        print(index)

a


{'Autauga, AL': {'n': 0, 'pop': 5586},
 'Baldwin, AL': {'n': 1, 'pop': 22323},
 'Barbour, AL': {'n': 2, 'pop': 2468},
 'Bibb, AL': {'n': 3, 'pop': 2239},
 'Blount, AL': {'n': 4, 'pop': 5782},
 'Bullock, AL': {'n': 5, 'pop': 1010},
 'Butler, AL': {'n': 6, 'pop': 1944},
 'Calhoun, AL': {'n': 7, 'pop': 11360},
 'Chambers, AL': {'n': 8, 'pop': 3325},
 'Cherokee, AL': {'n': 9, 'pop': 2619},
 'Chilton, AL': {'n': 10, 'pop': 4442},
 'Choctaw, AL': {'n': 11, 'pop': 1258},
 'Clarke, AL': {'n': 12, 'pop': 2362},
 'Clay, AL': {'n': 13, 'pop': 1323},
 'Cleburne, AL': {'n': 14, 'pop': 1491},
 'Coffee, AL': {'n': 15, 'pop': 5234},
 'Colbert, AL': {'n': 16, 'pop': 5524},
 'Conecuh, AL': {'n': 17, 'pop': 1206},
 'Coosa, AL': {'n': 18, 'pop': 1066},
 'Covington, AL': {'n': 19, 'pop': 3704},
 'Crenshaw, AL': {'n': 20, 'pop': 1377},
 'Cullman, AL': {'n': 21, 'pop': 8376},
 'Dale, AL': {'n': 22, 'pop': 4917},
 'Dallas, AL': {'n': 23, 'pop': 3719},
 'DeKalb, AL': {'n': 24, 'pop': 7151},
 'Elmore, AL': {'n'

In [62]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
    
with open('../src/counties.json', 'w') as outfile:
    json.dump(a, outfile, cls=NumpyArrayEncoder)

In [63]:
import numpy as np

b = []

def gl(index):
    u = dfa.index.get_loc(index)
    if hasattr(u, "__len__"):
        u = np.where(u == True)[0][0]
    return u

for index, row in dfa.iterrows():
    b.append({
        'pos': gl(index),
        'pop': int(row[2019]/10)
    })

    # get_loc gibt array of boolean zurück, wenn key mehrdeutig ist
    # lösung: dfa.reset_index()?

b = sorted(b, key=lambda k: k['pop'], reverse=True)

print(', '.join(list(map(lambda k: str(k['pos']), b))))

175, 578, 2591, 74, 193, 186, 333, 2547, 1819, 189, 1716, 1836, 2937, 192, 2710, 2505, 296, 199, 1280, 157, 1826, 1193, 2262, 190, 340, 1847, 318, 1798, 338, 1825, 2035, 2717, 1308, 1261, 2028, 2213, 2762, 163, 2816, 1949, 1917, 417, 1175, 77, 2533, 166, 1546, 342, 1855, 713, 305, 3055, 279, 2474, 424, 1743, 584, 1810, 1176, 2947, 171, 280, 2551, 194, 1248, 2598, 283, 212, 2561, 2257, 1198, 1163, 1753, 2951, 2041, 2201, 2569, 1197, 1748, 2153, 1189, 325, 1016, 197, 195, 390, 401, 1823, 231, 343, 236, 1195, 290, 1498, 611, 2414, 661, 1763, 1750, 36, 1239, 217, 2170, 2282, 2769, 2220, 1754, 2660, 1756, 901, 295, 2209, 1184, 2736, 245, 1162, 1649, 2234, 1187, 288, 1761, 341, 354, 206, 1343, 3027, 2247, 2087, 1898, 607, 2067, 2226, 2306, 2952, 1196, 215, 942, 1745, 1757, 205, 1755, 833, 2926, 709, 519, 1838, 349, 1729, 2860, 2442, 1191, 210, 78, 1829, 2917, 2278, 204, 198, 1744, 1097, 183, 348, 1106, 1300, 2058, 2521, 2217, 2178, 1737, 2323, 2840, 48, 2293, 1223, 3082, 330, 1542, 187, 141,